# PPG Signal Analysis

### Import packages

In [ ]:
import neurokit2 as nk
from scipy import signal
import scipy.stats as stats
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
from utils.load_data import load_csv_data_ppg
# %matplotlib tk
%matplotlib inline


### Specify data path and initialize variables

In [ ]:
sampling_rate = 250.0
data_path = r'sample_data'
filepath = os.path.join(data_path, 'P1_S1_baseline_1674758272_855629.csv')

### Load Data

In [ ]:
ppg1, ppg2, event_code = load_csv_data_ppg(filepath)

In [ ]:
time_axis = np.arange(len(ppg1))/sampling_rate
fig, ax = plt.subplots(2, 1, layout='tight')
ax[0].plot(time_axis, ppg1)
ax[1].plot(time_axis, ppg2)

### Resample the signal utilizing arduino_ts - interpolate

In [ ]:
# resample the signals here

### Signal Filtering and Visualization

In [ ]:
sos = signal.butter(2, (0.5, 5.0), 'bandpass', fs=sampling_rate, output='sos')
filtered_ppg1 = signal.sosfilt(sos, ppg1)
filtered_ppg2 = signal.sosfilt(sos, ppg2)


In [ ]:
fig, ax = plt.subplots(2, 1)
ax[0].plot(time_axis, filtered_ppg1, 'g')
# plt.plot(arduino_ts[event_code != -1], ppg1[event_code != -1], 'ro', markersize=2)
ax[0].set_xlabel("Time (seconds)")
ax[0].set_ylabel("PPG-Finger")

ax[1].plot(time_axis, filtered_ppg2, 'b')
# plt.plot(arduino_ts[event_code != -1], ppg2[event_code != -1], 'ro', markersize=2)
ax[1].set_xlabel("Time (seconds)")
ax[1].set_ylabel("PPG-Ear")


### Process PPG Signals - NeuroKit Library

In [ ]:
signals1, info1 = nk.ppg_process(filtered_ppg1, sampling_rate=sampling_rate)
plt.plot(np.arange(0, len(signals1['PPG_Clean'])/sampling_rate, 1/sampling_rate), signals1['PPG_Clean'])
plt.plot(info1['PPG_Peaks']/sampling_rate, signals1['PPG_Clean'][info1['PPG_Peaks']], 'go')
plt.xlabel("Time (seconds)")
plt.ylabel("Normalized Signal Amplitude")

In [ ]:
signals2, info2 = nk.ppg_process(filtered_ppg2, sampling_rate=sampling_rate)
plt.plot(np.arange(0, len(signals2['PPG_Clean'])/sampling_rate, 1/sampling_rate), signals2['PPG_Clean'])
plt.plot(info2['PPG_Peaks']/sampling_rate, signals2['PPG_Clean'][info2['PPG_Peaks']], 'go')
plt.xlabel("Time (seconds)")
plt.ylabel("Normalized Signal Amplitude")

In [ ]:
print(vars(signals1))

In [ ]:
print(info1)

In [ ]:
plt.plot(np.arange(len(signals1['PPG_Rate']))/sampling_rate, signals1['PPG_Rate'], 'b', label='PPG-Finger')
plt.plot(np.arange(len(signals2['PPG_Rate']))/sampling_rate, signals2['PPG_Rate'], 'g', label='PPG-Ear')
plt.legend()
plt.xlabel("Time (seconds)")
plt.ylabel("Pulse Rate")


### Correlation Analysis Between PPG-Finger and PPG-Ear

In [ ]:
r, p = stats.pearsonr(signals1['PPG_Rate'], signals2['PPG_Rate'])
print(f"Computed Pearson r: {r} and p-value: {p}")


### HRV Features

In [ ]:
hrv_indices1 = nk.hrv(info1['PPG_Peaks'], sampling_rate=250, show=True)


In [ ]:
hrv_indices2 = nk.hrv(info2['PPG_Peaks'], sampling_rate=250, show=True)


In [ ]:
for i, nm in enumerate(hrv_indices1):
    val1 = hrv_indices1.get(nm)
    val2 = hrv_indices2.get(nm)
    if (val1.values[0] == val1.values[0]) and (val2.values[0] == val2.values[0]):
        print(nm, "\t\t", val1.values[0], "\t\t", val2.values[0])
